# ABcache
## getting a precomputed ABcache file containing the parameters for beta-binomial distribution from the PoN-list

# Running EBfilter createCache on testdata

## setting the config

In [1]:
cd ..

/Users/martinscience/mount/work/utils/EBFilter


In [ ]:
from codes import run
import os
HOME = os.environ['HOME'] 
# HOME = f"{HOME}/mount" # set HOME to run on different Macs
# get configs
log = 'output/logfile'
sep = 'tab'
_q = 20  # mapping quality=20
_Q = 15 # base quality=15
fit_pen = 0.5
filter_quals = ''
for qual in range( 33, 33 + _Q ): 
    filter_quals += chr( qual )  # qual asciis for filtering out
_ff = 'UNMAP,SECONDARY,QCFAIL,DUP'     # default
config = {'q':_q, 'Q':_Q, 'filter_quals': filter_quals, 'log':log, 'fitting_penalty': fit_pen, 'ff':_ff, 'sep': sep}
args = {}

## running makeEBcache on testdata (takes ~2h on 3 cores)
* is not necessary for testing the cache mode
* a precomputed EBcache is stored in testdata/precom_testcache

In [ ]:
args['pon_list'] = 'testdata/PoN_list.txt'
args['cache_folder'] = 'testdata/testdata_cache' # provide a folder for storing the data (snakemake config)
args['force_caching'] = True               # force cache generation although no bed_file is provided
args['bed_file'] = None # 'testdata/input.bed'
args['generate_cache'] = True

In [ ]:
config['threads'] = 4
config['debug_mode'] = False
run.main(args, config)
!ls output

### using CLI:

In [ ]:
!./makeEBcache -t 3 -force_caching testdata/PoN_list.txt testdata/testdata_cache

## running EBscore in cache_mode on testdata
* you can use the cache file generated in the last step or the precomputed one
* for using precomputed file, just change the args['use_cache'] value

In [ ]:
args['mut_file'] = 'testdata/input.anno'
args['tumor_bam'] = 'testdata/tumor.bam'
args['pon_list'] = 'testdata/PoN_list.txt'
args['output_path'] = 'output/testdata_eb.csv'
# toggle outcomment in the next two lines for use of precomputed cache file
args['use_cache'] = 'testdata/testdata_cache' 
# args['use_cache'] = 'testdata/precom_testcache'
args['chromosome'] = 'chr11'
args['generate_cache'] = False

In [ ]:
config['threads'] = 3
config['debug_mode'] = False
run.main(args, config)
!ls output

### using CLI:

In [ ]:
!EBscore -t 3 -use_cache testdata/testdata_cache testdata/input.anno testdata/tumor.bam testdata/PoN_list.txt output/testdata_EB.csv

## Running EBfilter createCache on my data (takes ~5h on 3 cores)

In [ ]:
args = {}
args['pon_list'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_pon.list'
args['cache_folder'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_cache'
args['force_caching'] = False
args['bed_file'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/HAEv7.bed'
args['chrom'] = '22'
args['generate_cache'] = True

In [ ]:
config['threads'] = 3
config['debug_mode'] = False
run.main(args, config)
!ls output

### using CLI:

In [ ]:
!makeEBcache -t3 -force_caching ~/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_pon.list ~/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_cache

## Running EBscore in cache_mode on my data

In [ ]:
args['mut_file'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/anno/test_rel.csv'
args['tumor_bam'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/bam/test_Rel1.bam'
args['pon_list'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_pon.list'
args['use_cache'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_cache'
args['output_path'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/output/test_eb.csv'
args['chromosome'] = 'chr11'
args['generate_cache'] = False

In [ ]:
config['threads'] = 40
config['debug_mode'] = True
run.main(args, config)

### using CLI:

In [ ]:
!EBscore -t 3 -use_cache ~/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_cache ~//Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/anno/test_rel.csv ~/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/bam/test_Rel1.bam ~/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_pon.list ~/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/output/test_eb.csv


## Running EBfilter createCache in BIHCluster on real data

In [ ]:
from codes import run
import os
HOME = os.environ['HOME']
# HOME = f"{HOME}/mount" # set HOME to run on different Macs
STATIC = f'{HOME}/work/static'
# STATIC = '/fast/groups/ag_damm/work/static'
args = {'q':20, 'Q':15, 'fit_with': 0.5, 'ff':'UNMAP,SECONDARY,QCFAIL,DUP', 'sep': 'tab'}
filter_quals = ''
for qual in range( 33, 33 + args['Q'] ): 
    filter_quals += chr( qual )  # qual asciis for filtering out   
args['filter_quals'] = filter_quals

args['pon_list'] = f'{STATIC}/ref/PoN/AML_Pon.txt'
args['cache_folder'] = f'{STATIC}/ref/PoN/AML_cache'
args['force_caching'] = False
args['chrom'] = 'Y'
args['bed_file'] = f'{STATIC}/ref/bed_files/SureSelect/hg19/SS_HAEv6r2_hg19_Padded_nochr.bed'
args['generate_cache'] = True

args['t'] = 32
args['debug'] = False
run.main(args, make_cache=True)

### using CLI:

In [ ]:
!makeEBcache -t3 -bed_file ~/work/static/ref/bed_files/SureSelect/hg19/SS_HAEv6r2_hg19_Padded_nochr.bed ~/work/static/ref/PoN/AML_Pon.txt ~/work/static/ref/PoN/AML_cache


## Running EBscore in cache_mode in BIHCluster on real data

In [ ]:
from codes import run
import os
HOME = os.environ['HOME']
STATIC = f'{HOME}/work/static'
# STATIC = '/fast/groups/ag_damm/work/static'
# HOME = f"{HOME}/mount" # set HOME to run on different Macs
args = {'q':20, 'Q':15, 'fit_with': 0.5, 'ff':'UNMAP,SECONDARY,QCFAIL,DUP', 'sep': 'tab'}
filter_quals = ''
for qual in range( 33, 33 + args['Q'] ): 
    filter_quals += chr( qual )  # qual asciis for filtering out   
args['filter_quals'] = filter_quals
args['mut_file'] = f'{HOME}/scratch/projects/somVar/annovar/test_D-CR1.csv'
args['output_path'] = f'{HOME}/work/utils/EBFilter/output/test_D-EB.csv'
args['tumor_bam'] = f'{HOME}/scratch/projects/somVar/recalib/test_D.bam'
args['pon_list'] = f'{STATIC}/ref/PoN/AML_Pon.txt'
args['use_cache'] = f'{STATIC}/ref/PoN/AML_cache'
args['generate_cache'] = False
args['region'] = ''

args['t'] = 1
args['debug'] = True
run.main(args, make_cache=False)

+ customized smaller version for debugging

In [ ]:
from codes import run
import os
HOME = os.environ['HOME']
STATIC = f'{HOME}/work/static'
# STATIC = '/fast/groups/ag_damm/work/static'
# HOME = f"{HOME}/mount" # set HOME to run on different Macs
args = {'q':20, 'Q':15, 'fit_with': 0.5, 'ff':'UNMAP,SECONDARY,QCFAIL,DUP', 'sep': 'tab'}
filter_quals = ''
for qual in range( 33, 33 + args['Q'] ): 
    filter_quals += chr( qual )  # qual asciis for filtering out   
args['filter_quals'] = filter_quals
args['mut_file'] = f'{HOME}/work/utils/EBFilter/test_D-Indel.csv'
args['output_path'] = f'{HOME}/work/utils/EBFilter/output/test_D-EB.csv'
args['tumor_bam'] = f'{HOME}/scratch/projects/somVar/recalib/test_D.bam'
args['pon_list'] = f'{STATIC}/ref/PoN/AML_Pon.txt'
args['use_cache'] = f'{STATIC}/ref/PoN/AML_cache'
args['generate_cache'] = False
args['region'] = ''

args['t'] = 1
args['debug'] = True
run.main(args, make_cache=False)